# Capture odds - election winner at sportsbet

Note: this notebook is for ease of testing. Convert to a python file and move to the automated directory.

To do this ...
```
jupyter nbconvert --to python _Capture-odds-election-winner-at-sportsbet.ipynb

chmod 700 _Capture-odds-election-winner-at-sportsbet.py

mv _Capture-odds-election-winner-at-sportsbet.py ../automated
```

Shell file called by cron:
```
#!/bin/zsh
#crontab: 57 11 * * * /Users/bryanpalmer/Australian-Federal-Election-2025/automated/election-winner.sh

# set-up parameters
home=/Users/bryanpalmer
working=Australian-Federal-Election-2025/automated
runrun=_Capture-odds-election-winner-at-sportsbet.py
mmenv=p311

# move to the home directory
cd $home

# activate the micromamba environment
micromamba activate $mmenv

# move to the working directory
cd $working

#initiate the data capture
$home/micromamba/envs/$mmenv/bin/python ./$runrun >>log.log 2>> err.log
```

## Python setup

In [1]:
# data science imports
# System imports
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# --- web scraping imports
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager

print(f"webdriver_manager version: {webdriver_manager.__version__}")

webdriver_manager version: 4.0.2


## Set-up web-driver options

## Extract website text using Selenium

In [2]:
service = ChromeService()
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--headless")
driver = webdriver.Chrome(
    # July 2024 - webdriver-manager 4.0.1 is broken (installed 4.0.2 via pip)
    # See here: https://stackoverflow.com/questions/78789819/issue-with-selenium-and-webdriver
    # Earlier problem described here: https://www.youtube.com/watch?v=9vD8XBfY9yw
    options=options, service=ChromeService(ChromeDriverManager().install())

    # Earlier temporary fix for selenium issues.
    #options=options, service=service
)

driver.implicitly_wait(5)
url = (
    #"https://www.sportsbet.com.au/betting/politics/australian-federal-politics/"
    #+ "48th-parliament-of-australia-6560505"
    
    # updated 3 Nov 2024
    "https://www.sportsbet.com.au/betting/politics/australian-federal-politics/"
    + "48th-parliament-of-australia-8571604"
)
driver.get(url)
soup = BeautifulSoup(driver.page_source, "lxml")
driver.close()

In [3]:
print(soup.prettify())

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   48th Parliament of Australia betting odds - Australian Federal Politics | Sportsbet
  </title>
  <meta content="View the latest odds on 48th Parliament of Australia Matches &amp; Bet with Sportsbet. Join Australia's Favourite Online Betting and Entertainment Website." data-automation-id="page-description" data-react-helmet="true" name="description"/>
  <meta content="summary_large_image" data-react-helmet="true" property="twitter:card"/>
  <meta content="https://www.sportsbet.com.au/betting/politics/australian-federal-politics/48th-parliament-of-australia-8571604" data-react-helmet="true" property="twitter:url"/>
  <meta content="https://www.sportsbet.com.au" data-react-helmet="true" property="twitter:domain"/>
  <meta content="48th Parliament of Australia betting odds - Australian Federal Politics" data-react-helmet="true" property="twitter:title"/>
  <meta content="View the latest odds on 48th Parliament of Australia Mat

## Extract data of interest

In [4]:
# Step one - get to the right bit of the page ...
# <div class="background_fja218n" data-automation-id="content-background">
div_name = "content-background"  # this looks fragile
div = soup.find_all("div", {"data-automation-id": div_name})[0]
# print(div.prettify())

In [5]:
# step 2
# <div class="outcomeCardItems_f4kk892">
odds = div.find_all("div", {"class": "outcomeCardItems_f4kk892"})  # fragile
len(odds)  # should be two: for a names row and a prices row

2

In [6]:
found = {}
for k, v in zip(odds[0].contents, odds[1].contents):
    found[k.text] = v.text
found  # should be a dictionary of parties and prices/odds

{'Coalition': '1.83', 'Labor': '2.00', 'Any Other Result': '81.00'}

## Append this data to a CSV file

In [7]:
# long format
df = pd.DataFrame([found.keys(), found.values()], index=["variable", "value"]).T
df.index = np.repeat(datetime.datetime.now(), len(df))
df.index.name = "datetime"

In [8]:
df

,variable,value
datetime,,
2024-11-07 10:42:36.056004,Coalition,1.83
2024-11-07 10:42:36.056004,Labor,2.00
2024-11-07 10:42:36.056004,Any Other Result,81.00


In [9]:
# save to file
DIR = "../betting-data"
Path(DIR).mkdir(parents=True, exist_ok=True)
FILE = f"{DIR}/sportsbet-2025-election-winner.csv"
df.to_csv(FILE, mode="a", index=True, header=False)

## Done